In [5]:
from mp_api.client import MPRester

In [6]:
# API_KEYを読み込む
with open("MaterialsAPI_key.txt", "r") as file: # MaterialsAPI_key.txt へのパスは自分の環境に合わせて変更する
    API_KEY = file.read()

In [7]:
#API_KEYの設定
mpr = MPRester(API_KEY)

In [8]:
# battery explorerから取得できるデータの一覧
mpr.materials.insertion_electrodes.available_fields

['battery_type',
 'battery_id',
 'thermo_type',
 'battery_formula',
 'working_ion',
 'num_steps',
 'max_voltage_step',
 'last_updated',
 'framework',
 'framework_formula',
 'elements',
 'nelements',
 'chemsys',
 'formula_anonymous',
 'warnings',
 'formula_charge',
 'formula_discharge',
 'max_delta_volume',
 'average_voltage',
 'capacity_grav',
 'capacity_vol',
 'energy_grav',
 'energy_vol',
 'fracA_charge',
 'fracA_discharge',
 'stability_charge',
 'stability_discharge',
 'id_charge',
 'id_discharge',
 'host_structure',
 'adj_pairs',
 'material_ids',
 'entries_composition_summary',
 'electrode_object']

In [3]:
import pandas as pd

df = pd.read_csv("clean_Li_battery_data.csv")

In [13]:
# 取得したデータ数
df.shape

(2440, 26)

In [5]:
from sqlalchemy import create_engine

In [2]:
db_user = 'valley3'
db_password_file = './postgres_password.txt'
db_host = 'postgres'  # Dockerコンテナ内のPostgreSQLサービス名を指定します。
db_port = '5432'  # PostgreSQLのデフォルトポートは5432です。
db_name = 'material_db'

In [10]:
# PostgreSQLデータベースに接続
# パスワードをファイルから読み込む
with open(db_password_file, 'r') as f:
    db_password = f.read().strip()

# PostgreSQLの接続URIを作成
db_uri = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# SQLAlchemyを使用してエンジンを作成し、DataFrameをPostgreSQLに挿入
engine = create_engine(db_uri)
df.to_sql('material_db', con=engine, if_exists='replace', index=False)

# 接続を閉じる
engine.dispose()

In [11]:
# PostgreSQLの接続URIを作成
db_uri = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# SQLAlchemyを使用してエンジンを作成し、データベースからデータを読み込む
engine = create_engine(db_uri)

# データを読み込むSQLクエリを作成（ここでは全てのデータを読み込む例）
sql_query = "SELECT * FROM material_db"

# SQLクエリを実行し、データをDataFrameとして読み込む
df_from_db = pd.read_sql(sql_query, con=engine)

# 接続を閉じる
engine.dispose()

# 読み込んだデータを確認する
df_from_db

,battery_formula,num_steps,max_voltage_step,framework,framework_formula,elements,nelements,chemsys,formula_anonymous,formula_charge,...,energy_vol,fracA_charge,fracA_discharge,stability_charge,stability_discharge,host_structure,adj_pairs,entries_composition_summary,electrode_object,fields_not_requested
0,Li2-4CoNi3O8,2,0.221134,"{'Co': 1.0, 'Ni': 3.0, 'O': 8.0}",CoNi3O8,"['Co', 'Ni', 'O']",3,Co-Ni-O,AB3C8,Li2CoNi3O8,...,2349.032129,0.142857,0.250000,0.037008,0.010849,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li2CoNi3O8', 'formula_dis...","{'all_formulas': ['Li4CoNi3O8', 'Li2CoNi3O8', ...",{'voltage_pairs': [{'voltage': 3.7556553933333...,"['battery_type', 'thermo_type']"
1,Li3-4CrNi3O8,1,0.000000,"{'Cr': 1.0, 'Ni': 3.0, 'O': 8.0}",CrNi3O8,"['Cr', 'Ni', 'O']",3,Cr-Ni-O,AB3C8,Li3CrNi3O8,...,1126.234230,0.200000,0.250000,0.089504,0.038122,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Li3CrNi3O8', 'formula_dis...","{'all_formulas': ['Li4CrNi3O8', 'Li3CrNi3O8'],...",{'voltage_pairs': [{'voltage': 3.5559580133333...,"['battery_type', 'thermo_type']"
2,Li0-4Cu3Sb(PO4)4,1,0.000000,"{'Cu': 3.0, 'Sb': 1.0, 'P': 4.0, 'O': 16.0}",Cu3Sb(PO4)4,"['Cu', 'Sb', 'P', 'O']",4,Cu-O-P-Sb,AB3C4D16,Cu3Sb(PO4)4,...,2083.089272,0.000000,0.142857,0.067563,0.072026,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Cu3Sb(PO4)4', 'formula_di...","{'all_formulas': ['Li4Cu3Sb(PO4)4', 'Cu3Sb(PO4...",{'voltage_pairs': [{'voltage': 3.6375136033333...,"['battery_type', 'thermo_type']"
3,Li1-2Cu(HO)4,1,0.000000,"{'Cu': 1.0, 'H': 4.0, 'O': 4.0}",Cu(HO)4,"['Cu', 'H', 'O']",3,Cu-H-O,AB4C4,LiCu(HO)4,...,1503.189668,0.100000,0.181818,0.078001,0.080676,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'LiCu(HO)4', 'formula_disc...","{'all_formulas': ['Li2Cu(HO)4', 'LiCu(HO)4'], ...",{'voltage_pairs': [{'voltage': 2.9016335933333...,"['battery_type', 'thermo_type']"
4,Li0-4Mn3CuNi2(PO4)6,1,0.000000,"{'Mn': 3.0, 'Cu': 1.0, 'Ni': 2.0, 'P': 6.0, 'O...",Mn3CuNi2(PO4)6,"['Mn', 'Cu', 'Ni', 'P', 'O']",5,Cu-Mn-Ni-O-P,AB2C3D6E24,Mn3CuNi2(PO4)6,...,1452.065776,0.000000,0.100000,0.045275,0.082965,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'Mn3CuNi2(PO4)6', 'formula...","{'all_formulas': ['Mn3CuNi2(PO4)6', 'Li4Mn3CuN...",{'voltage_pairs': [{'voltage': 3.7576910933333...,"['battery_type', 'thermo_type']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,Li0-0.12VO2,1,0.000000,"{'V': 1.0, 'O': 2.0}",VO2,"['V', 'O']",2,O-V,AB2,VO2,...,382.085393,0.000000,0.040000,0.137107,0.130874,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'VO2', 'formula_discharge'...","{'all_formulas': ['VO2', 'LiV8O16'], 'all_chem...",{'voltage_pairs': [{'voltage': 2.7833067433333...,"['battery_type', 'thermo_type']"
2436,Li0-2V3O7,1,0.000000,"{'V': 3.0, 'O': 7.0}",V3O7,"['V', 'O']",2,O-V,A3B7,V3O7,...,1890.562742,0.000000,0.166667,0.082159,0.067151,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'V3O7', 'formula_discharge...","{'all_formulas': ['Li2V3O7', 'V3O7'], 'all_che...",{'voltage_pairs': [{'voltage': 3.2735610533333...,"['battery_type', 'thermo_type']"
2437,Li0-1V2O5,1,0.000000,"{'V': 2.0, 'O': 5.0}",V2O5,"['V', 'O']",2,O-V,A2B5,V2O5,...,1554.658080,0.000000,0.125000,0.012942,0.025422,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'V2O5', 'formula_discharge...","{'all_formulas': ['V2O5', 'LiV2O5'], 'all_chem...",{'voltage_pairs': [{'voltage': 3.4979915308333...,"['battery_type', 'thermo_type']"
2438,Li0-0.67V2O5,2,0.641710,"{'V': 2.0, 'O': 5.0}",V2O5,"['V', 'O']",2,O-V,A2B5,V2O5,...,1114.284551,0.000000,0.086957,0.029818,0.034136,"{'@module': 'pymatgen.core.structure', '@class...","[{'formula_charge': 'V2O5', 'formula_discharge...","{'all_formulas': ['Li2V6O15', 'V2O5', 'LiV4O10...",{'voltage_pairs': [{'voltage': 3.6917310599999...,"['battery_type', '

In [ ]:
# データ確認
df.shape

In [ ]:
# データ情報
df.info()

In [ ]:
# データ保存
clean_df.to_csv("Li_battery_data.csv",index=False)